In [12]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Flatten,Dense
import cv2
import keras
#cv2.resize(s, (224,224))
#target_f=0
EPISODES = 1000

class DQNAgent:
    #def __init__(self, state_size, action_size):
    def __init__(self, action_size):
        #self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        vgg16_model=keras.applications.vgg16.VGG16()
        vgg16_model.layers.pop()
        model=keras.Sequential()
        for layer in vgg16_model.layers:
            model.add(layer)
        for layer in model.layers:
            layer.trainable=False
        model.add(Dense(32,activation="relu"))
        model.add(Dense(4,activation="linear"))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = np.expand_dims(state, axis=0)############
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state = np.expand_dims(next_state, axis=0)#######
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)))
            state = np.expand_dims(state, axis=0)########################
            target_f = self.model.predict(state)
            print(target_f)
            target_f[0][action] = target
            print(target_f,"vishal",target)
            #np.reshape(target_f,(1,5))
            print(target_f,"Vishal Poddar")
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    #def load(self, name):
    #    self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [13]:
def car_action(temp):
    val=[0]*4
    val[temp]=1
    return [0.8*(val[0]-val[1]),val[2],val[3]]

In [14]:
if __name__ == "__main__":
    env = gym.make("CarRacing-v0")
    #state_size = env.observation_space.shape[0]
    #action_size = env.action_space.n
    action_size=4
    agent = DQNAgent(action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state=cv2.resize(state, (224,224))
        #state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            a=agent.act(state)
            action = car_action(a)
            next_state, reward, done, _ = env.step(action)
            next_state=cv2.resize(next_state, (224,224))
            reward = reward if not done else -10
            #next_state = np.reshape(next_state, [1, state_size])
            agent.memorize(state, a, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

Track generation: 1282..1607 -> 325-tiles track
[[ 0.717928   1.0602539 -0.869491   0.6606239]]
[[ 0.717928   1.0602539 -0.869491   6.9723988]] vishal 6.972398592734042
[[ 0.717928   1.0602539 -0.869491   6.9723988]] Vishal Poddar
[[ 1.2089401  -0.11046309  0.41048673  3.023284  ]]
[[ 2.3505123  -0.11046309  0.41048673  3.023284  ]] vishal 2.3505122780799863
[[ 2.3505123  -0.11046309  0.41048673  3.023284  ]] Vishal Poddar
[[ 3.647333   1.2551079 -1.0150247  5.733023 ]]
[[3.647333  1.2551079 3.4823909 5.733023 ]] vishal 3.482390797138214
[[3.647333  1.2551079 3.4823909 5.733023 ]] Vishal Poddar
[[ 5.3719134e+00 -1.8755421e-03  1.7257473e+00  6.7799621e+00]]
[[ 5.3719134e+00 -1.8755421e-03  1.7257473e+00  6.8373270e+00]] vishal 6.837327218055725
[[ 5.3719134e+00 -1.8755421e-03  1.7257473e+00  6.8373270e+00]] Vishal Poddar
[[6.975323  1.2663035 2.2346926 9.724877 ]]
[[8.02525   1.2663035 2.2346926 9.724877 ]] vishal 8.025250768661499
[[8.02525   1.2663035 2.2346926 9.724877 ]] Vishal Pod

[[21.730534 21.809536 20.811438 22.367083]]
[[22.519312 21.809536 20.811438 22.367083]] vishal 22.51931285858154
[[22.519312 21.809536 20.811438 22.367083]] Vishal Poddar
[[22.515    20.805882 19.441717 19.811472]]
[[21.818184 20.805882 19.441717 19.811472]] vishal 21.818183040618898
[[21.818184 20.805882 19.441717 19.811472]] Vishal Poddar
[[20.752903 21.314795 20.319147 22.278595]]
[[20.752903 21.314795 20.319147 21.489939]] vishal 21.489939022064206
[[20.752903 21.314795 20.319147 21.489939]] Vishal Poddar
[[22.58638  23.573423 21.645592 24.154554]]
[[21.177858 23.573423 21.645592 24.154554]] vishal 21.177857875823975
[[21.177858 23.573423 21.645592 24.154554]] Vishal Poddar
[[20.047367 20.229876 16.559374 22.521624]]
[[21.344229 20.229876 16.559374 22.521624]] vishal 21.34422855377197
[[21.344229 20.229876 16.559374 22.521624]] Vishal Poddar
[[20.707481 20.537306 17.209219 22.73894 ]]
[[23.157053 20.537306 17.209219 22.73894 ]] vishal 23.15705232620239
[[23.157053 20.537306 17.2092

[[23.725119 21.493532 23.670427 20.505953]]
[[23.725119 22.294128 23.670427 20.505953]] vishal 22.294128894805905
[[23.725119 22.294128 23.670427 20.505953]] Vishal Poddar
[[22.155537 20.139805 21.602694 17.782845]]
[[22.155537 20.139805 22.911888 17.782845]] vishal 22.911887645721436
[[22.155537 20.139805 22.911888 17.782845]] Vishal Poddar
[[20.894766 19.503616 22.455702 18.498573]]
[[20.894766 22.005611 22.455702 18.498573]] vishal 22.00561075210571
[[20.894766 22.005611 22.455702 18.498573]] Vishal Poddar
[[21.623144 19.585342 22.442661 18.874958]]
[[21.623144 19.585342 21.701357 18.874958]] vishal 21.70135736465454
[[21.623144 19.585342 21.701357 18.874958]] Vishal Poddar
[[20.901983 20.037085 18.868639 16.439827]]
[[20.901983 20.037085 18.868639 20.912516]] vishal 20.912514972686765
[[20.901983 20.037085 18.868639 20.912516]] Vishal Poddar
[[22.126488 20.98631  22.894339 20.179356]]
[[22.126488 20.98631  22.894339 20.258104]] vishal 20.258103466033937
[[22.126488 20.98631  22.894

[[22.000486 20.660078 21.877726 19.026342]]
[[22.000486 20.660078 21.877726 20.094309]] vishal 20.094309425354005
[[22.000486 20.660078 21.877726 20.094309]] Vishal Poddar
[[23.023785 21.231388 22.762167 20.692144]]
[[22.325825 21.231388 22.762167 20.692144]] vishal 22.325824069976804
[[22.325825 21.231388 22.762167 20.692144]] Vishal Poddar
[[22.004736 21.045145 21.944277 19.38074 ]]
[[22.004736 20.747227 21.944277 19.38074 ]] vishal 20.747226047515866
[[22.004736 20.747227 21.944277 19.38074 ]] Vishal Poddar
[[19.286312 19.503492 20.047241 18.186426]]
[[19.286312 19.503492 20.047241 20.09377 ]] vishal 20.093769454956053
[[19.286312 19.503492 20.047241 20.09377 ]] Vishal Poddar
[[23.61669  23.079735 22.406027 20.446802]]
[[23.61669  21.730349 22.406027 20.446802]] vishal 21.73034906387329
[[23.61669  21.730349 22.406027 20.446802]] Vishal Poddar
[[21.72054  21.080484 21.643505 20.014917]]
[[20.74431  21.080484 21.643505 20.014917]] vishal 20.744310569763186
[[20.74431  21.080484 21.64

[[13.053239 13.164509 15.620137 14.187717]]
[[13.78577  13.164509 15.620137 14.187717]] vishal 13.785770034790039
[[13.78577  13.164509 15.620137 14.187717]] Vishal Poddar
[[15.589045 13.23064  15.744345 14.412372]]
[[15.589045 13.23064  15.744345 13.955413]] vishal 13.955413150787352
[[15.589045 13.23064  15.744345 13.955413]] Vishal Poddar
[[15.309845 12.893727 15.662402 13.584503]]
[[15.309845 12.893727 15.056147 13.584503]] vishal 15.056146430969237
[[15.309845 12.893727 15.056147 13.584503]] Vishal Poddar
[[16.021894  13.0287075 16.096443  14.475423 ]]
[[16.021894  13.0287075 14.600233  14.475423 ]] vishal 14.600232934951782
[[16.021894  13.0287075 14.600233  14.475423 ]] Vishal Poddar
[[15.506226 12.665623 15.818039 13.77816 ]]
[[14.88308  12.665623 15.818039 13.77816 ]] vishal 14.883079576492309
[[14.88308  12.665623 15.818039 13.77816 ]] Vishal Poddar
[[15.118042 14.217408 14.480257 13.963946]]
[[15.118042 14.780762 14.480257 13.963946]] vishal 14.780761528015136
[[15.118042 14

...

[[31.951265 34.986137 34.350475 32.61428 ]]
[[31.865215 34.986137 34.350475 32.61428 ]] vishal 31.86521472930908
[[31.865215 34.986137 34.350475 32.61428 ]] Vishal Poddar
[[32.373714 32.356724 33.5141   31.539701]]
[[32.373714 32.356724 33.5141   30.753433]] vishal 30.75343379974365
[[32.373714 32.356724 33.5141   30.753433]] Vishal Poddar
[[23.663124 21.292923 22.800352 20.4506  ]]
[[23.663124 21.470034 22.800352 20.4506  ]] vishal 21.470034408569333
[[23.663124 21.470034 22.800352 20.4506  ]] Vishal Poddar
[[29.881706 30.672714 31.414522 28.404757]]
[[29.881706 30.672714 29.828516 28.404757]] vishal 29.82851524353027
[[29.881706 30.672714 29.828516 28.404757]] Vishal Poddar
[[30.522282 31.526377 32.034588 29.264221]]
[[30.67447  31.526377 32.034588 29.264221]] vishal 30.674471282958983
[[30.67447  31.526377 32.034588 29.264221]] Vishal Poddar
[[29.600151 30.388094 31.052347 27.915253]]
[[29.600151 30.749067 31.052347 27.915253]] vishal 30.74906692504883
[[29.600151 30.749067 31.05234

[[24.792076 23.032806 23.772148 23.715538]]
[[24.792076 23.514196 23.772148 23.715538]] vishal 23.514195632934566
[[24.792076 23.514196 23.772148 23.715538]] Vishal Poddar
[[25.470863 24.719246 25.624052 27.701508]]
[[25.470863 24.719246 26.477037 27.701508]] vishal 26.477036762237546
[[25.470863 24.719246 26.477037 27.701508]] Vishal Poddar
[[19.95436  16.927723 19.139763 17.752956]]
[[19.95436  17.89482  19.139763 17.752956]] vishal 17.894819736480713
[[19.95436  17.89482  19.139763 17.752956]] Vishal Poddar
[[25.975693 27.532059 27.148222 26.764729]]
[[24.354866 27.532059 27.148222 26.764729]] vishal 24.35486612319946
[[24.354866 27.532059 27.148222 26.764729]] Vishal Poddar
[[26.005362 28.0405   27.770655 27.544477]]
[[26.005362 28.0405   27.770655 26.343407]] vishal 26.343406772613523
[[26.005362 28.0405   27.770655 26.343407]] Vishal Poddar
[[26.940104 28.722351 28.880981 27.503931]]
[[26.940104 28.722351 27.559977 27.503931]] vishal 27.559976387023923
[[26.940104 28.722351 27.55

[[21.39322  18.574747 19.956934 18.45293 ]]
[[21.39322  18.574747 20.921814 18.45293 ]] vishal 20.921814060211183
[[21.39322  18.574747 20.921814 18.45293 ]] Vishal Poddar
[[22.265175 19.181892 20.730574 19.164513]]
[[22.265175 19.181892 20.730574 21.060133]] vishal 21.060133457183838
[[22.265175 19.181892 20.730574 21.060133]] Vishal Poddar
[[23.6841   23.111862 22.923668 23.90902 ]]
[[23.6841   23.91572  22.923668 23.90902 ]] vishal 23.915719795227048
[[23.6841   23.91572  22.923668 23.90902 ]] Vishal Poddar
[[19.462158 16.391075 17.286236 15.75874 ]]
[[19.462158 16.391075 18.03302  15.75874 ]] vishal 18.033019542694092
[[19.462158 16.391075 18.03302  15.75874 ]] Vishal Poddar
[[22.194723 21.85169  21.780128 21.800146]]
[[21.180555 21.85169  21.780128 21.800146]] vishal 21.180555915832517
[[21.180555 21.85169  21.780128 21.800146]] Vishal Poddar
[[22.638327 19.357779 21.25466  19.803324]]
[[22.638327 19.357779 22.731201 19.803324]] vishal 22.731200504302976
[[22.638327 19.357779 22.7

[[15.667383 16.927357 16.87335  15.948174]]
[[15.667383 16.364649 16.87335  15.948174]] vishal 16.364648723602294
[[15.667383 16.364649 16.87335  15.948174]] Vishal Poddar
[[18.389244 18.835192 18.833843 19.141161]]
[[18.389244 18.835192 18.833843 17.461355]] vishal 17.461355781555174
[[18.389244 18.835192 18.833843 17.461355]] Vishal Poddar
[[19.322037 18.516102 19.446295 18.998848]]
[[18.20458  18.516102 19.446295 18.998848]] vishal 18.204579734802245
[[18.20458  18.516102 19.446295 18.998848]] Vishal Poddar
[[15.939288 15.37099  16.803709 14.776789]]
[[15.939288 15.37099  15.609802 14.776789]] vishal 15.609802722930908
[[15.939288 15.37099  15.609802 14.776789]] Vishal Poddar
[[13.841975 14.703893 14.46396  14.244689]]
[[13.841975  14.703893  13.8375635 14.244689 ]] vishal 13.837563705444335
[[13.841975  14.703893  13.8375635 14.244689 ]] Vishal Poddar
[[15.319953 14.656273 16.324453 13.497028]]
[[15.319953 14.656273 16.324453 15.255912]] vishal 15.255911922454834
[[15.319953 14.656

KeyboardInterrupt: 

In [17]:
 for e in range(10):
        state = env.reset()
        state=cv2.resize(next_state, (224,224))
        #state = np.reshape(state, [1, state_size])
        
        for time in range(500):
            env.render()
            action = agent.act(state)
            action = car_action(action)
            next_state, reward, done, _ = env.step(action)
            next_state=cv2.resize(next_state, (224,224))
            reward = reward if not done else -10
            #next_state = np.reshape(next_state, [1, state_size])
            #agent.memorize(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break

Track generation: 1211..1525 -> 314-tiles track
Track generation: 999..1258 -> 259-tiles track


KeyboardInterrupt: 

In [81]:
agent.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [18]:
env.close()